In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

torch.autograd.backward() 自动求取梯度 params：

tensors：用于求导的张量，如loss

retain_graph:保存计算图

create_graph:创建导数计算图，用于高阶求导

grad_tensors:多维度权值

In [9]:
w = torch.tensor([1.],requires_grad=True)
x = torch.tensor([2.],requires_grad=True)

a = torch.add(w,x)
b = torch.add(w,1)

y = torch.mul(a,b)
y.backward()
y.backward()
print(w.grad)

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

点进y.backward()方法中会发现tensor.backward() 他就是调用了torch.autograd.backward()方法

In [14]:
w = torch.tensor([1.],requires_grad=True)
x = torch.tensor([2.],requires_grad=True)

a = torch.add(w,x)
b = torch.add(w,1)

y = torch.mul(a,b)
y.backward(retain_graph=True)
print(w.grad)
y.backward()
print(w.grad)

tensor([5.])
tensor([10.])


In [24]:
w = torch.tensor([1.],requires_grad=True)
x = torch.tensor([2.],requires_grad=True)

a = torch.add(w,x)
b = torch.add(w,1)

y0 = torch.mul(a,b)
y0.retain_grad()
y1 = torch.add(a,b)
y1.retain_grad()

loss = torch.cat([y0,y1],dim=0)

grad_tensors = torch.tensor([1.,1.])
#grad_tensors = torch.tensor([1.,2.])  结果就变成9了  
loss.backward(gradient=grad_tensors)

print(w.grad)

#对y0和y1求导 其中y0对w的导数是5 y1对w的导数是2 之后和梯度权重grad_tensors相乘
#就是5*1 + 2*1 
#结果返回的是y0和y1综合起来的梯度

tensor([7.])


torch.autograd.grad 求取梯度 params：

outputs：用于求导的张量 如loss

input：需要求梯度的张量

create_graph:创建导数计算图 用于高阶求导

retain_graph:保存计算图

grad_outputs:多梯度权重

In [30]:
x = torch.tensor([3.],requires_grad=True)
y = torch.pow(x,2)

#只有创建了导数的计算图才能继续求高阶导数
grad_1 = torch.autograd.grad(y,x,create_graph=True)

grad_2 = torch.autograd.grad(grad_1[0],x,create_graph=True)
#grad_2 = torch.autograd.grad(grad_1,x)

#如果grad_2中不设置create_graph=True grad_3会报错
grad_3 = torch.autograd.grad(grad_2,x)

grad_1,grad_2,grad_3

((tensor([6.], grad_fn=<MulBackward0>),),
 (tensor([2.], grad_fn=<MulBackward0>),),
 (tensor([0.]),))

autograd：

1.梯度不自动清零 所以之前连续反向传播两次梯度叠加了

2.依赖于叶子结点的节点 requires_grad默认True

3.叶子结点不可执行in-place

In [33]:
w = torch.tensor([1.],requires_grad=True)
x = torch.tensor([2.],requires_grad=True)

for i in range(4):
    a = torch.add(w,x)
    b = torch.add(w,1)
    y = torch.mul(a,b)

    y.backward()
    print(w.grad)

tensor([5.])
tensor([10.])
tensor([15.])
tensor([20.])


In [35]:
w = torch.tensor([1.],requires_grad=True)
x = torch.tensor([2.],requires_grad=True)

for i in range(4):
    a = torch.add(w,x)
    b = torch.add(w,1)
    y = torch.mul(a,b)

    y.backward()
    print(w.grad)
#zero_ 这个_下划线表示原地操作
    w.grad.zero_()

tensor([5.])
tensor([5.])
tensor([5.])
tensor([5.])


In [36]:
w = torch.tensor([1.],requires_grad=True)
x = torch.tensor([2.],requires_grad=True)

a = torch.add(w,x)
b = torch.add(w,1)
y = torch.mul(a,b)

a.requires_grad,b.requires_grad,y.requires_grad

(True, True, True)

In [38]:
w = torch.tensor([1.],requires_grad=True)
x = torch.tensor([2.],requires_grad=True)

a = torch.add(w,x)
b = torch.add(w,1)
y = torch.mul(a,b)

w.add_(1)

y.backward()

RuntimeError: a leaf Variable that requires grad is being used in an in-place operation.

In [40]:
a = torch.ones(1,)
print(id(a),a)

a = a + torch.ones(1,)
print(id(a),a)

1909173143408 tensor([1.])
1909170238736 tensor([2.])


a = a + 1 操作之后 变量的内存地址发生变化 会开辟新的内存地址

这就是in-place 也就是原地操作

In [41]:
a = torch.ones(1,)
print(id(a),a)

a += torch.ones(1,)
print(id(a),a)

1909170632992 tensor([1.])
1909170632992 tensor([2.])


而+=操作是原地操作

叶子节点不能进行原地操作 

因为在反向传播的时候 是可能要用到叶子张量的值的 前向传播的时候是记录地址

反向传播是根据地址寻找 如果反向传播之前 改变了该地址的数据 那么梯度求解就会出错

这就是为什么叶子节点不能进行in-place操作 